Distreate feature generation

In [25]:
from scipy.sparse import csr_matrix

def one_hot_array(length, i, dense=False):
    if dense:
        rtn = np.zeros((1, length), dtype=np.float)
        if np.isscalar(i):
            rtn[0, i]=1
            return rtn
        else:
            for itm in i:
                rtn[0, itm]=1
            return rtn
    else:
        if np.isscalar(i):
            return csr_matrix(([1], ([0], [i])), dtype=np.float, shape=(1, length))
        else:
            return csr_matrix(([1]*len(i), ([0]*len(i), i)), dtype=np.float, shape=(1, length))
    

Generate features for a year

In [26]:
import pandas as pd
import numpy as np

def feature_generator(year):
    tourney_seeds = pd.read_csv("./Data/TourneySeeds.csv")
    rank = []
    teams = []
    for seed, team in tourney_seeds[tourney_seeds.Season==year].loc[:,['Seed','Team']].values:
        rank.append(int(seed[1:3]))
        teams.append(team-1101)
        
    f_team = dict({x: one_hot_array(365, x) for x in teams})
    
    f_rank = dict({x: one_hot_array(16, rank[i]-1) for i, x in enumerate(teams)})
    
    pretour_rank = []
    first_four_teams = []
    pretour_teams = []
    for seed, team in tourney_seeds[tourney_seeds.Season==(year-1)].loc[:,['Seed','Team']].values:
        pretour_rank.append(int(seed[1:3]))
        if len(seed)>3:
            first_four_teams.append(team-1101)
        pretour_teams.append(team-1101)
    t_dict = dict({x: one_hot_array(16, pretour_rank[i]-1) for i, x in enumerate(pretour_teams)})
    f_pretour_rank = ({
        x: t_dict[x] if x in t_dict else csr_matrix((1, 16), dtype=np.float) for x in teams
    })
    
    tour_result_simp = pd.read_csv("./Data/TourneyCompactResults.csv")
    tour_arc = np.hstack([tour_result_simp[tour_result_simp.ix[:, 0]==(year-1)].loc[:, 'Wteam'].values, 
                      tour_result_simp[tour_result_simp.ix[:, 0]==(year-1)].loc[:, 'Lteam'].values])
    tour_arc-=1101
    f_pretour_round = dict()
    unique, counts = np.unique(tour_arc, return_counts=True)
    pretour_cnt = dict(zip(unique, counts))
    for x in teams:
        if x in first_four_teams:
            f_pretour_round[x] = one_hot_array(7, range(0, pretour_cnt[x]))
        elif x in pretour_cnt:
            f_pretour_round[x] = one_hot_array(7, range(1, pretour_cnt[x]+1), True)
        else:
            f_pretour_round[x] = np.zeros((1, 7), dtype=np.float)
    
    tour_result = pd.read_csv("./Data/TourneyDetailedResults.csv")
    pair_pretour_result = dict()
    for row in tour_result[tour_result.loc[:, "Season"]==(year-1)].iterrows():
        rec = row[1]
        winner = rec['Wteam']-1101
        loser = rec['Lteam']-1101

        # on the winner's side
        v = np.zeros((1, 21), dtype=np.float)
        v[0, 0]=1 # winner
        if rec['Wscore']-rec['Lscore']<=3: v[0, 2]=1 # marginal win
        if rec['Numot']!=0: v[0, 4]=1 
        if rec['Wfgm']>rec['Lfgm']: v[0, 5]=1
        if rec['Wfga']>rec['Lfga']: v[0, 6]=1
        if rec['Wfgm']/(rec['Wfga']+0.0) > rec['Lfgm']/(rec['Lfga']+0.0): v[0, 7]=1
        if rec['Wfgm3']>rec['Lfgm3']: v[0, 8]=1
        if rec['Wfga3']>rec['Lfga3']: v[0, 9]=1
        if rec['Wfgm3']/(rec['Wfga3']+0.0) > rec['Lfgm3']/(rec['Lfga3']+0.0): v[0, 10]=1
        if rec['Wftm']>rec['Lftm']: v[0, 11]=1
        if rec['Wfta']>rec['Lfta']: v[0, 12]=1
        if rec['Wftm']/(rec['Wfta']+0.0) > rec['Lftm']/(rec['Lfta']+0.0): v[0, 13]=1
        if rec['Wor']>rec['Lor']: v[0, 14]=1
        if rec['Wdr']>rec['Ldr']: v[0, 15]=1
        if rec['Wast']>rec['Last']: v[0, 16]=1
        if rec['Wto']>rec['Lto']: v[0, 17]=1
        if rec['Wstl']>rec['Lstl']: v[0, 18]=1
        if rec['Wblk']>rec['Lblk']: v[0, 19]=1
        if rec['Wpf']>rec['Lpf']: v[0, 20]=1

        if winner not in pair_pretour_result:
            pair_pretour_result[winner] = []
        pair_pretour_result[winner].append((loser, v))

        # on the loser's side
        v = np.zeros((1, 21), dtype=np.float)
        v[0, 1]=1 # loser
        if rec['Wscore']-rec['Lscore']<=3: v[0, 3]=1 # marginal lose
        if rec['Numot']!=0: v[0, 4]=1 
        if rec['Wfgm']<rec['Lfgm']: v[0, 5]=1
        if rec['Wfga']<rec['Lfga']: v[0, 6]=1
        if rec['Wfgm']/(rec['Wfga']+0.0) < rec['Lfgm']/(rec['Lfga']+0.0): v[0, 7]=1
        if rec['Wfgm3']<rec['Lfgm3']: v[0, 8]=1
        if rec['Wfga3']<rec['Lfga3']: v[0, 9]=1
        if rec['Wfgm3']/(rec['Wfga3']+0.0) < rec['Lfgm3']/(rec['Lfga3']+0.0): v[0, 10]=1
        if rec['Wftm']<rec['Lftm']: v[0, 11]=1
        if rec['Wfta']<rec['Lfta']: v[0, 12]=1
        if rec['Wftm']/(rec['Wfta']+0.0) < rec['Lftm']/(rec['Lfta']+0.0): v[0, 13]=1
        if rec['Wor']<rec['Lor']: v[0, 14]=1
        if rec['Wdr']<rec['Ldr']: v[0, 15]=1
        if rec['Wast']<rec['Last']: v[0, 16]=1
        if rec['Wto']<rec['Lto']: v[0, 17]=1
        if rec['Wstl']<rec['Lstl']: v[0, 18]=1
        if rec['Wblk']<rec['Lblk']: v[0, 19]=1
        if rec['Wpf']<rec['Lpf']: v[0, 20]=1
        if loser not in pair_pretour_result:
            pair_pretour_result[loser] = []
        pair_pretour_result[loser].append((winner, v))
    f_pretour_record = dict()
    for team in teams:
        if team not in pair_pretour_result:
            f_pretour_record[team] = csr_matrix((1, 365*21), dtype = np.float)
        else:
            record = sorted(pair_pretour_result[team], key=lambda x:x[0])
            col=[]
            for base, arr in record:
                col.extend(list(base*21+arr.nonzero()[1]))
            f_pretour_record[team] = one_hot_array(365*21, col)
            
    regular_result = pd.read_csv("./Data/RegularSeasonDetailedResults.csv")
    pair_regular_result = dict()
    for row in regular_result[regular_result.loc[:, "Season"]==year].iterrows():
        rec = row[1]
        winner = rec['Wteam']-1101
        loser = rec['Lteam']-1101

        # on the winner's side
        v = np.zeros((1, 22), dtype=np.float)
        v[0, 0]=1 # winner
        if rec['Wscore']-rec['Lscore']<=3: v[0, 2]=1 # marginal win
        if rec['Numot']!=0: v[0, 4]=1 
        if rec['Wfgm']>rec['Lfgm']: v[0, 5]=1
        if rec['Wfga']>rec['Lfga']: v[0, 6]=1
        if rec['Wfgm']/(rec['Wfga']+1e-9) > rec['Lfgm']/(rec['Lfga']+1e-9): v[0, 7]=1
        if rec['Wfgm3']>rec['Lfgm3']: v[0, 8]=1
        if rec['Wfga3']>rec['Lfga3']: v[0, 9]=1
        if rec['Wfgm3']/(rec['Wfga3']+1e-9) > rec['Lfgm3']/(rec['Lfga3']+1e-9): v[0, 10]=1
        if rec['Wftm']>rec['Lftm']: v[0, 11]=1
        if rec['Wfta']>rec['Lfta']: v[0, 12]=1
        if rec['Wftm']/(rec['Wfta']+1e-9) > rec['Lftm']/(rec['Lfta']+1e-9): v[0, 13]=1
        if rec['Wor']>rec['Lor']: v[0, 14]=1
        if rec['Wdr']>rec['Ldr']: v[0, 15]=1
        if rec['Wast']>rec['Last']: v[0, 16]=1
        if rec['Wto']>rec['Lto']: v[0, 17]=1
        if rec['Wstl']>rec['Lstl']: v[0, 18]=1
        if rec['Wblk']>rec['Lblk']: v[0, 19]=1
        if rec['Wpf']>rec['Lpf']: v[0, 20]=1
        if rec['Wloc']=='H': 
            v[0, 21]=1
        elif rec['Wloc']=='A':
            v[0, 21]=2


        if winner not in pair_regular_result:
            pair_regular_result[winner] = []
        pair_regular_result[winner].append((loser, v))

        # on the loser's side
        v = np.zeros((1, 22), dtype=np.float)
        v[0, 1]=1 # loser
        if rec['Wscore']-rec['Lscore']<=3: v[0, 3]=1 # marginal lose
        if rec['Numot']!=0: v[0, 4]=1 
        if rec['Wfgm']<rec['Lfgm']: v[0, 5]=1
        if rec['Wfga']<rec['Lfga']: v[0, 6]=1
        if rec['Wfgm']/(rec['Wfga']+1e-9) < rec['Lfgm']/(rec['Lfga']+1e-9): v[0, 7]=1
        if rec['Wfgm3']<rec['Lfgm3']: v[0, 8]=1
        if rec['Wfga3']<rec['Lfga3']: v[0, 9]=1
        if rec['Wfgm3']/(rec['Wfga3']+1e-9) < rec['Lfgm3']/(rec['Lfga3']+1e-9): v[0, 10]=1
        if rec['Wftm']<rec['Lftm']: v[0, 11]=1
        if rec['Wfta']<rec['Lfta']: v[0, 12]=1
        if rec['Wftm']/(rec['Wfta']+1e-9) < rec['Lftm']/(rec['Lfta']+1e-9): v[0, 13]=1
        if rec['Wor']<rec['Lor']: v[0, 14]=1
        if rec['Wdr']<rec['Ldr']: v[0, 15]=1
        if rec['Wast']<rec['Last']: v[0, 16]=1
        if rec['Wto']<rec['Lto']: v[0, 17]=1
        if rec['Wstl']<rec['Lstl']: v[0, 18]=1
        if rec['Wblk']<rec['Lblk']: v[0, 19]=1
        if rec['Wpf']<rec['Lpf']: v[0, 20]=1
        if rec['Wloc']=='H': 
            v[0, 21]=2
        elif rec['Wloc']=='A':
            v[0, 21]=1
        if loser not in pair_regular_result:
            pair_regular_result[loser] = []
        pair_regular_result[loser].append((winner, v))
        
    f_regular_record = dict()
    for team in teams:
        if team not in pair_regular_result:
            f_regular_record[team] = csr_matrix((1, 365*21*3), dtype = np.float)
        else:
            record = sorted(pair_regular_result[team], key=lambda x:x[0])
            col=[]
            for base, arr in record:
                col.extend(list(base*21*3+arr[0, 21]*21+arr[0, :21].nonzero()[0]))

            f_regular_record[team] = one_hot_array(365*21*3, sorted(col))
            
    return (teams, (f_team, f_rank, f_pretour_rank, f_pretour_round, f_pretour_record, f_regular_record))

In [27]:
def cross_features(*args):
    mtxlen = [m.shape[1] for m in args]
    totallen = reduce(lambda x, y: x*y, mtxlen)
    acc = []
    cur = totallen
    for itm in mtxlen:
        acc.append(cur/itm)
        cur = acc[-1]
    
    col = []
    from itertools import product
    for itm in product(*[m.indices for m in args]):
        v = 0
        for i, c in enumerate(itm):
            v+=acc[i]*c
        col.append(v)
    return one_hot_array(totallen, col)

In [15]:
cross_features(fs[0][2], fs[1][2])

<1x5840 sparse matrix of type '<type 'numpy.int32'>'
	with 1 stored elements in Compressed Sparse Row format>

Generate training data

In [28]:
def get_feature_for_pair(p, y):
    # get feature of pairlst p in year y 
    X = []
    teamlst, (f_team, f_rank, f_pretour_rank, f_pretour_round, f_pretour_record, f_regular_record) = feature_generator(y)
    
    from scipy.sparse import hstack
    for pair in p:
        X.append(hstack([
                f_team[pair[0]],
                f_team[pair[1]],
                f_rank[pair[0]],
                f_rank[pair[1]],
                f_pretour_record[pair[0]],
                f_regular_record[pair[0]],
                f_pretour_record[pair[1]],
                f_regular_record[pair[1]],
                cross_features(f_team[pair[0]], f_team[pair[1]]),
                cross_features(f_team[pair[0]], f_rank[pair[1]]),
                cross_features(f_team[pair[1]], f_rank[pair[0]]),
                cross_features(f_rank[pair[0]], f_rank[pair[1]])
            ]))
    return X

In [14]:
X = get_feature_for_pair(pair, year)

In [ ]:
import pandas as pd
import numpy as np


tcr = pd.read_csv("./Data/TourneyCompactResults.csv")
years = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
X = []
Y = []
for year in years:
    y = []
    pair = []
    for w, l in tcr[tcr.Season==year].loc[:, ['Wteam', 'Lteam']].values:
        pair.extend([(w-1101, l-1101), (l-1101, w-1101)])
        y.extend([1, 0])
    x = get_feature_for_pair(pair, year)
    Y.extend(y)
    X.extend(x)

In [16]:
vyears = [2013]
Xv = []
Yv = []
for year in vyears:
    y = []
    pair = []
    for w, l in tcr[tcr.Season==year].loc[:, ['Wteam', 'Lteam']].values:
        pair.extend([(w-1101, l-1101), (l-1101, w-1101)])
        y.extend([1, 0])
    x = get_feature_for_pair(pair, year)
    Yv.extend(y)
    Xv.extend(x)

In [22]:
from scipy.sparse import vstack
X = vstack(X).tocsr()
Xv = vstack(Xv).tocsr()
Y = np.asarray(Y, dtype=np.float)
Yv = np.asarray(Yv, dtype = np.float)

In [23]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X, Y)
lgb_valid = lgb.Dataset(Xv, Yv, reference=lgb_train)

In [20]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'tree_learner': 'feature',
    'num_threads': 4,
    'metric': 'binary_logloss',
    'num_leaves': 127,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

In [24]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_valid,
                early_stopping_rounds=5)

TypeError: Expected np.float32 or np.float64, met type(int32)

<1164x207243 sparse matrix of type '<type 'numpy.int32'>'
	with 770130 stored elements in Compressed Sparse Row format>